In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Configs

In [3]:
LABEL2IDX = {
    'O': 0,
    'B-DAT': 1,
    'B-PER': 2,
    'B-ORG': 3,
    'B-LOC': 4,
    'B-EVE': 5,
    'I-DAT': 6,
    'I-PER': 7,
    'I-ORG': 8,
    'I-LOC': 9,
    'I-EVE': 10
}

IDX2LABEL = {i: k for k, i in LABEL2IDX.items()}

MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
EPOCHS = 8
NUM_CLASS = 11
LEARNING_RATE = 1e-5

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import concatenate_datasets, load_dataset
from transformers import AutoTokenizer
import ast
model_name = 'sbunlp/fabert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_files = "/content/drive/MyDrive/Colab Notebooks/NER-datasets/shuffled-100000-2.csv"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
def parse_tokens_and_labels(example):
    # Ensure 'tokens' and 'labels' are in the expected format
    if isinstance(example['tokens'], str):
        example['tokens'] = ast.literal_eval(example['tokens'])
    if isinstance(example['labels'], str):
        example['labels'] = ast.literal_eval(example['labels'])
    return example


In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],truncation=True,max_length=512,
 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [8]:
dataset = load_dataset('csv', data_files=data_files)
dataset = dataset['train']
# Apply the parsing function to the entire dataset
parsed_dataset = dataset.map(parse_tokens_and_labels)
# rename labels to net_tags
parsed_dataset = parsed_dataset.rename_column("labels", "ner_tags")
tokenized_dataset = parsed_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=parsed_dataset.column_names,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100000
})

In [10]:
# Printing the values and lengths of each column in the first item of tokenized_dataset
print(f"Input IDs: {tokenized_dataset[0]['input_ids']}, length: {len(tokenized_dataset[0]['input_ids'])}")
print(f"Attention Mask: {tokenized_dataset[0]['attention_mask']}, length: {len(tokenized_dataset[0]['attention_mask'])}")
print(f"Labels: {tokenized_dataset[0]['labels']}, length: {len(tokenized_dataset[0]['labels'])}")

Input IDs: [101, 5055, 4588, 43143, 2303, 2588, 3087, 2342, 2307, 3570, 49470, 2697, 2297, 5042, 34007, 15941, 6663, 2342, 622, 2297, 5042, 2588, 17710, 48332, 2307, 43143, 8080, 2303, 2402, 2352, 3775, 3249, 4588, 2549, 622, 2491, 2303, 11707, 2307, 2902, 2601, 3610, 2303, 2402, 2445, 2297, 9477, 2293, 3793, 791, 2003, 2299, 3656, 34007, 15941, 2856, 622, 2314, 3249, 2297, 3845, 3845, 14467, 2342, 117, 102], length: 66
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], length: 66
Labels: [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 4, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, -100, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], length: 66


In [11]:
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][1]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-2]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-1]))

[CLS]
نحوه
.
[SEP]


In [12]:
print(tokenizer.convert_ids_to_tokens(tokenized_dataset[0]['input_ids']))


['[CLS]', 'نحوه', 'دعوت', 'پیرنیا', 'از', 'وی', 'چنین', 'بود', 'که', 'روزی', 'نجا', '##حی', 'در', 'منزل', 'عبدالعلی', 'وزیری', 'مهمان', 'بود', 'و', 'در', 'منزل', 'وی', 'سنتور', 'میڸزده', 'که', 'پیرنیا', 'بلافاصله', 'از', 'او', 'برای', 'اجرای', 'برنامه', 'دعوت', 'میڸکند', 'و', 'پس', 'از', 'امتحانی', 'که', 'توسط', 'چند', 'استاد', 'از', 'او', 'میڸشود', 'در', 'گله', '##ای', 'شماره', '۲', '##۰', 'به', 'اتفاق', 'عبدالعلی', 'وزیری', 'شرکت', 'و', 'این', 'برنامه', 'در', 'دستگاه', 'دستگاه', 'همایون', 'بود', '.', '[SEP]']


In [13]:
# Split the combined dataset into train, validation, and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_eval_split = train_test_split['test'].train_test_split(test_size=0.5)

train_dataset = train_test_split['train']
eval_dataset = train_eval_split['train']
test_dataset = train_eval_split['test']
# Print out info of each dataset
print(f"Training set: {train_dataset} samples")
print(f"Validation set: {eval_dataset} samples")
print(f"Test set: {test_dataset} samples")


Training set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 80000
}) samples
Validation set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10000
}) samples
Test set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10000
}) samples


In [14]:
from transformers import DataCollatorForTokenClassification
import torch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [15]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

In [16]:
train_sample = train_dataset[0]
print(type(train_sample['input_ids']))

<class 'torch.Tensor'>


In [17]:
# check the paddings with -100s
batch = data_collator([train_dataset[i] for i in range(2)])
batch["labels"]

tensor([[-100,    0, -100, -100, -100,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    3,    0, -100, -100,    0,    0,    0,    0,    0,    4,
         -100, -100,    0,    0,    0,    0,    0,    0,    0, -100, -100,    0,
            0,    0,    0, -100],
        [-100,    0,    0, -100, -100,    0,    4,    9,    0,    0,    0,    4,
            0,    0,    0,    0,    0,    0,    0, -100,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    4,    0,    0,
            0,    0, -100, -100]])

In [18]:
for i in range(2):
    print(train_dataset[i]["labels"])

tensor([-100,    0, -100, -100, -100,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    3,    0, -100, -100,    0,    0,    0,    0,    0,    4,
        -100, -100,    0,    0,    0,    0,    0,    0,    0, -100, -100,    0,
           0,    0,    0, -100])
tensor([-100,    0,    0, -100, -100,    0,    4,    9,    0,    0,    0,    4,
           0,    0,    0,    0,    0,    0,    0, -100,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    4,    0,    0,
           0,    0, -100])


In [ ]:
IDX2LABEL

{0: 'O',
 1: 'B-DAT',
 2: 'B-PER',
 3: 'B-ORG',
 4: 'B-LOC',
 5: 'B-EVE',
 6: 'I-DAT',
 7: 'I-PER',
 8: 'I-ORG',
 9: 'I-LOC',
 10: 'I-EVE'}

In [ ]:
LABEL2IDX

{'O': 0,
 'B-DAT': 1,
 'B-PER': 2,
 'B-ORG': 3,
 'B-LOC': 4,
 'B-EVE': 5,
 'I-DAT': 6,
 'I-PER': 7,
 'I-ORG': 8,
 'I-LOC': 9,
 'I-EVE': 10}

In [19]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=88e3ffc5a107ae017d7de8da8f3da599827f2b3c1beff731bd894dbccb44f970
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [20]:
label_list = [k for k,v in LABEL2IDX.items()]
label_list

['O',
 'B-DAT',
 'B-PER',
 'B-ORG',
 'B-LOC',
 'B-EVE',
 'I-DAT',
 'I-PER',
 'I-ORG',
 'I-LOC',
 'I-EVE']

In [21]:
import numpy as np
from seqeval.metrics import classification_report
import evaluate

accuracy = evaluate.load('accuracy')
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #############
    cr = classification_report(true_labels, true_predictions,digits=4)
    fname = "report.txt"
    file = open(fname,'w')
    file.write(cr)
    file.close()
    #############
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [22]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig
model_name = 'pouria82/faBERT-armanpeyma-48'

config = AutoConfig.from_pretrained(model_name)
config.attention_probs_dropout_prob = 0.2  #  default (0.1)
config.hidden_dropout_prob = 0.2  # default (0.1)

model = AutoModelForTokenClassification.from_pretrained(model_name, config=config)

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [23]:
samples = train_dataset[:8]
batch = data_collator([train_dataset[i] for i in range(i)])
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([1, 40]),
 'token_type_ids': torch.Size([1, 40]),
 'attention_mask': torch.Size([1, 40]),
 'labels': torch.Size([1, 40])}

In [24]:
 model.config.max_position_embeddings

512

In [25]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 80000
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})


In [26]:
LEARNING_RATE

1e-05

In [27]:
model.name_or_path

'pouria82/faBERT-armanpeyma-48'

In [29]:
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    "faBERT-armanpeyma-finetuned-4880",
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size= TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= EVAL_BATCH_SIZE,
    num_train_epochs= 8,
    weight_decay= 0.025,#*********************
    warmup_ratio = 0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    lr_scheduler_type="linear",
    greater_is_better=True,
    push_to_hub=True,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.158000,0.157792,0.643733,0.715505,0.677724,0.941765
2,0.138400,0.156538,0.655535,0.720830,0.686634,0.943676
3,0.123300,0.169311,0.646954,0.733636,0.687574,0.941875
4,0.105500,0.173224,0.650961,0.727394,0.687058,0.942126
5,0.091700,0.183139,0.663643,0.722069,0.691625,0.943860
6,0.077700,0.196936,0.667932,0.711007,0.688796,0.944092
7,0.068100,0.204481,0.659645,0.720141,0.688567,0.943285


TrainOutput(global_step=70000, training_loss=0.10904016898018973, metrics={'train_runtime': 6398.9629, 'train_samples_per_second': 100.016, 'train_steps_per_second': 12.502, 'total_flos': 1.7274741981303504e+16, 'train_loss': 0.10904016898018973, 'epoch': 7.0})

In [30]:
model.config.num_labels

11

In [31]:
validation_results = trainer.evaluate()

In [32]:
validation_results

{'eval_loss': 0.1831391602754593,
 'eval_precision': 0.663643266959163,
 'eval_recall': 0.722069218764344,
 'eval_f1': 0.6916245328643658,
 'eval_accuracy': 0.9438604873387482,
 'eval_runtime': 29.7984,
 'eval_samples_per_second': 335.589,
 'eval_steps_per_second': 41.949,
 'epoch': 7.0}

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
        )
test_results = trainer.evaluate()

In [34]:
test_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [35]:
test_results

{'eval_loss': 0.17629604041576385,
 'eval_precision': 0.6709209633649932,
 'eval_recall': 0.7268259072117593,
 'eval_f1': 0.6977554350222692,
 'eval_accuracy': 0.9452059594115334,
 'eval_runtime': 31.0935,
 'eval_samples_per_second': 321.611,
 'eval_steps_per_second': 40.201}

In [36]:
with open('report.txt') as f:
    for line in f.readlines():
        print(line)

              precision    recall  f1-score   support



         DAT     0.7613    0.7902    0.7755      7516

         EVE     0.4899    0.6005    0.5396       771

         LOC     0.6266    0.6615    0.6436      6565

         ORG     0.6199    0.7005    0.6578      2952

         PER     0.6585    0.7590    0.7052      3966



   micro avg     0.6709    0.7268    0.6978     21770

   macro avg     0.6313    0.7023    0.6643     21770

weighted avg     0.6732    0.7268    0.6986     21770

